In [14]:
!pip install map-boxes
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
from tqdm import tqdm
import pandas as pd
import random

/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.7/dist-packages/map_boxes/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [16]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 11806, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 11806 (delta 1), reused 0 (delta 0), pack-reused 11800
Receiving objects: 100% (11806/11806), 12.41 MiB | 24.82 MiB/s, done.
Resolving deltas: 100% (8115/8115), done.


In [17]:
!pip install -r /content/yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 27.2 MB/s 
     |████████████████████████████████| 1.6 MB 47.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [19]:
path_to_train = '/content/drive/MyDrive/data_sec/datasets/train' # путь до тренировочных данных
path_to_test = '/content/drive/MyDrive/data_sec/datasets/test' # путь до валидационных данных
path_to_train_imgs = f'{path_to_train}/images/'
path_to_test_imgs = f'{path_to_test}/images/'
path_to_train_labels = f'{path_to_train}/labels/'

In [22]:
dict_objects = {'car': [], 'human': [], 'head': [], 'face': [], 'carplate': []}
dict_cat_objects = {'car': 0, 'head': 1, 'face': 2, 'human': 3, 'carplate': 4}

In [10]:
for file_name in os.listdir(path_to_train_labels): # Разделение тренировочных labels на классы
    if 'car' in file_name and 'carplate' not in file_name:
        dict_objects['car'].append(file_name)
    elif 'human' in file_name:
        dict_objects['human'].append(file_name)
    elif 'head' in file_name:
        dict_objects['head'].append(file_name)
    elif 'carplate' in file_name:
        dict_objects['carplate'].append(file_name)
    elif 'face' in file_name:
        dict_objects['face'].append(file_name)

In [11]:
dict_objects['head'][0]

'26-11-2021_04-47-50_AM_head..txt'

In [12]:
#распределение объектов в train
pd.Series([file_name.split('_')[-1] for file_name in os.listdir(path_to_train_labels)]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [13]:
#создадим обучающий датасет в формате yolo
!mkdir data_for_yolo
!mkdir data_for_yolo/data
!mkdir data_for_yolo/data/images
!mkdir data_for_yolo/data/labels
!mkdir data_for_yolo/data/images/train
!mkdir data_for_yolo/data/labels/train
!mkdir data_for_yolo/data/images/test
!mkdir data_for_yolo/data/labels/test

In [14]:
names = ['car', 'head', 'face', 'human', 'carplate'] # список названий классов

In [15]:
# car: 512 size, 0.3 - iou_threshold
# carplate: 1024 size, 0.4 - iou_threshold

In [13]:
name = names[3] # текущий объект детекции

In [17]:
#в папке data_for_yolo создадим dataset.yaml со следующим содержимым

yaml_content = {
    'train': '/content/drive/MyDrive/data_sec/data_for_yolo/data/images/train', 
     'val': '/content/drive/MyDrive/data_sec/data_for_yolo/data/images/test',
     'nc': 1,
     'names': [name]
}

In [ ]:
# добавление в словарь labels в соответствии классам
dict_images_names = {'car': [], 'human': [], 'head': [], 'face': [], 'carplate': []}

for key in dict_objects:
    for val in dict_objects[key]:
        if 'car' in val and 'carplate' not in val:
            dict_images_names['car'].append(val.split('_car..txt')[0])
        elif 'human' in val:
            dict_images_names['human'].append(val.split('_human..txt')[0])
        elif 'head' in val:
            dict_images_names['head'].append(val.split('_head..txt')[0])
        elif 'carplate' in val:
            dict_images_names['carplate'].append(val.split('_carplate..txt')[0])
        elif 'face' in val:
            dict_images_names['face'].append(val.split('_face..txt')[0])


In [ ]:
train_images_car, test_images_car = train_test_split(dict_images_names['car'], test_size=0.2, random_state=22)
train_images_human, test_images_human = train_test_split(dict_images_names['human'], test_size=0.2, random_state=22)
train_images_head, test_images_head = train_test_split(dict_images_names['head'], test_size=0.2, random_state=22)
train_images_carplate, test_images_carplate = train_test_split(dict_images_names['carplate'], test_size=0.1, random_state=22)
train_images_face, test_images_face = train_test_split(dict_images_names['face'], test_size=0.1, random_state=22)

In [ ]:
# выбор train и val для обучения в сооветствии классу
train_images_name = train_images_car
test_images_name = test_images_car

In [ ]:
# Наполнение папок файлами для обучения

#train
for train_image in tqdm(train_images_name):
  #labels
  for file in os.listdir(path_to_train_labels): 
    if f'_{name}..txt' in file and train_image in file:
      rep_name = name
      shutil.copy(path_to_train_labels + file, '/content/drive/MyDrive/data_sec/data_for_yolo/data/labels/train/' + file.replace(f'_{rep_name}..txt', '.txt'))
  #images '/content/drive/MyDrive/data_sec/datasets/train'
  for file in os.listdir(path_to_train_imgs): 
    if train_image + '.jpg' == file: 
      shutil.copy(path_to_train_imgs + file, '/content/drive/MyDrive/data_sec/data_for_yolo/data/images/train/')

#test
for test_image in tqdm(test_images_name):
  #labels
  for file in os.listdir(path_to_train_labels): 
    if f'_{name}..txt' in file and test_image in file:
      rep_name = name
      shutil.copy(path_to_train_labels + file, '/content/drive/MyDrive/data_sec/data_for_yolo/data/labels/test/' + file.replace(f'_{rep_name}..txt', '.txt'))
  #images 
  for file in os.listdir(path_to_train_imgs): 
    if test_image + '.jpg' == file: 
      shutil.copy(path_to_train_imgs + file, '/content/drive/MyDrive/data_sec/data_for_yolo/data/images/test/')

100%|██████████| 63/63 [00:28<00:00,  2.23it/s]


In [18]:
print(len(os.listdir('/content/drive/MyDrive/data_sec/data_for_yolo/data/images/test')))
print(len(os.listdir('/content/drive/MyDrive/data_sec/data_for_yolo/data/images/train')))

348
1392


In [19]:
print(len(os.listdir('/content/drive/MyDrive/data_sec/data_for_yolo/data/labels/test')))
print(len(os.listdir('/content/drive/MyDrive/data_sec/data_for_yolo/data/labels/train')))

348
1392


### Добавление новых данных в data

In [ ]:
c = k = 0
for i in os.listdir('/content/drive/MyDrive/data_sec/data_for_yolo/data/labels/train'):
  if 'Part' in i:
    c += 1
    #os.remove('/content/drive/MyDrive/data_sec/data_for_yolo/data/labels/test/' + i)
  else:
    k += 1
print(c, k)

550 1392


In [ ]:
lst_names_txt_train = []
lst_names_txt_test = []
for c, i in enumerate(os.listdir(path_to_txt_files)):
  if c < 550:
    lst_names_txt_train.append(i.split('.')[0])
    shutil.copy(path_to_txt_files + '/' + i, '/content/drive/MyDrive/data_sec/data_for_yolo/data/labels/train/' + i)

for c, i in enumerate(os.listdir(path_to_txt_files)):
  if c > 749:
    lst_names_txt_test.append(i.split('.')[0])
    shutil.copy(path_to_txt_files + '/' + i, '/content/drive/MyDrive/data_sec/data_for_yolo/data/labels/test/' + i)

for c, i in enumerate(os.listdir('/content/drive/MyDrive/data_sec/data_for_yolo/data_without_new_data/images_2')):
  if i.split('.')[0] in lst_names_txt_train:
    shutil.copy('/content/drive/MyDrive/data_sec/data_for_yolo/data_without_new_data/images_2/' + i, '/content/drive/MyDrive/data_sec/data_for_yolo/data/images/train/' + i)

for c, i in enumerate(os.listdir('/content/drive/MyDrive/data_sec/data_for_yolo/data_without_new_data/images_2')):
  if i.split('.')[0] in lst_names_txt_test:
    shutil.copy('/content/drive/MyDrive/data_sec/data_for_yolo/data_without_new_data/images_2/' + i, '/content/drive/MyDrive/data_sec/data_for_yolo/data/images/test/' + i)

### Парсинг и преобразование xml-файлов в yolo-формат

In [ ]:
path_to_xml_files = '/content/drive/MyDrive/data_sec/data_for_yolo/data_without_new_data/labels_2_xml'
path_to_txt_files = '/content/drive/MyDrive/data_sec/data_for_yolo/data_without_new_data/labels_2_txt'

In [ ]:
from bs4 import BeautifulSoup
 
def parseXML(xml_file):
    fd = open(xml_file, 'r') 
    xml_file = fd.read() 
    soup = BeautifulSoup(xml_file, 'xml')
    lst = []
    for x_min, y_min, x_max, y_max in zip(soup.findAll('xmin'), soup.findAll('ymin'), soup.findAll('xmax'), soup.findAll('ymax')): 
      lst.append(list(map(float, [x_min.text, x_max.text, y_min.text, y_max.text])))
    width, height = int(soup.find('width').text), int(soup.find('height').text)
    fd.close()
    return width, height, lst


In [ ]:
def convert_coordinates(size, boxes):
    lst = []
    for box in boxes:
        dw = 1.0/size[0]
        dh = 1.0/size[1]
        x = (box[0]+box[1])/2.0
        y = (box[2]+box[3])/2.0
        w = box[1]-box[0]
        h = box[3]-box[2]
        x = x*dw
        w = w*dw
        y = y*dh
        h = h*dh
        lst.append((x,y,w,h))
    return lst

In [ ]:
for name_img in tqdm(os.listdir(path_to_xml_files)):
  width, height, coords = parseXML(f'{path_to_xml_files}/{name_img}')
  yolo_format_coords = convert_coordinates((width, height), coords)
  file_txt = f'{path_to_txt_files}/{name_img.split(".")[0]}.txt'
  file = open(file_txt, 'w')
  for coord in yolo_format_coords:
    #file.write(str(dict_cat_objects[name]) + ' ' + str(coord)[1:-1].replace(', ', ' ') + '\n')
    file.write('0' + ' ' + str(coord)[1:-1].replace(', ', ' ') + '\n')
    
  file.close()


In [ ]:
c = 0
for i in os.listdir(path_to_txt_files):
  c += 1
print(c)

### Обучение

In [20]:
path_to_data = f'/content/drive/MyDrive/data_sec/data_for_yolo/dataset_{name}.yaml'

In [21]:
#обучение
import yaml
with open(f'{path_to_data}', 'w') as f:
    yaml.dump(yaml_content, f)

In [ ]:
yaml_content

In [ ]:
!nvidia-smi

In [ ]:
!pip install wandb
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 20.4 MB/s 
     |████████████████████████████████| 181 kB 74.6 MB/s 
     |████████████████████████████████| 157 kB 69.7 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 157 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 74.4 MB/s 
     |████████████████████████████████| 157 kB 78.6 MB/s 
     |████████████████████████████████| 157 kB 74.3 MB/s 
     |████████████████████████████████| 156 kB 78.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=04a8632ebe6571c26b346e659c1cc61875537cee826358f5be7f26c27a7147b2
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public

In [ ]:
!python ./yolov5/train.py --img 1280 --batch -1 --epochs 300 --data {path_to_data} --weights /content/drive/MyDrive/data_sec/data_for_yolo/outputs/results_human/best_4.pt --cfg /content/yolov5/models/hub/yolov5x6.yaml  --name yolov5x_results --cache

train: weights=/content/drive/MyDrive/data_sec/data_for_yolo/outputs/results_face/best_1.pt, cfg=/content/yolov5/models/hub/yolov5x6.yaml, data=/content/drive/MyDrive/data_sec/data_for_yolo/dataset_face.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=-1, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5x_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-11-g529aafd Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, wa

### Предсказание

In [24]:
!python3 yolov5/detect.py --weights /content/drive/MyDrive/data_sec/data_for_yolo/outputs/results_human/best_4.pt --img 1280 --conf 0.4 --source /content/drive/MyDrive/data_sec/datasets/test/images --save-txt --save-conf

detect: weights=['/content/drive/MyDrive/data_sec/data_for_yolo/outputs/results_human/best_4.pt'], source=/content/drive/MyDrive/data_sec/datasets/test/images, data=yolov5/data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.2-17-ga5a47c5 Python-3.7.13 torch-1.12.1+cu113 CPU

Fusing layers... 
Model summary: 574 layers, 139970872 parameters, 0 gradients, 207.9 GFLOPs
image 1/865 /content/drive/MyDrive/data_sec/datasets/test/images/0_23-11-2021_02-18-50_PM.jpg: 896x1280 (no detections), 12234.8ms
image 2/865 /content/drive/MyDrive/data_sec/datasets/test/images/0_23-11-2021_02-18-56_PM.jpg: 1088x1280 1 person, 12306.8ms
image 3/865 /content/drive/MyDriv

In [25]:
def get_soliton_labels_df_simple_solution(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      val = [float(i) for i in line.split()]
      _, xywh, conf = val[0], val[1:5], val[5]
      center_x, center_y, width, height = xywh
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, dict_cat_objects[name], conf, xmin, xmax, ymin, ymax])
  return simple_solution

In [32]:
simple_solution = get_soliton_labels_df_simple_solution('yolov5/runs/detect/exp/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [33]:
simple_solution.to_csv("path/sample_solution.csv", sep=';', index=False)

In [34]:
df = pd.read_csv("path/sample_solution.csv", sep=';', index_col=None)

In [35]:
df

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_06-42-28_AM.jpg,4,0.902363,0.936719,0.951953,0.308333,0.392361
1,25-11-2021_06-59-00_PM.jpg,4,0.926120,0.234896,0.268230,0.396296,0.517592
2,26-11-2021_04-48-23_AM.jpg,4,0.882410,0.502084,0.536459,0.447222,0.594444
3,24-11-2021_12-56-12_PM.jpg,4,0.903053,0.109003,0.126489,0.338158,0.427632
4,26-11-2021_04-49-01_AM.jpg,4,0.882026,0.792187,0.806771,0.198148,0.272222
...,...,...,...,...,...,...,...
1922,25-11-2021_09-46-29_AM.jpg,4,0.901243,0.756250,0.776562,0.216667,0.327777
1923,24-11-2021_07-53-46_AM.jpg,4,0.494995,0.869141,0.885547,0.309722,0.380556
1924,24-11-2021_07-53-46_AM.jpg,4,0.898202,0.593750,0.618360,0.365278,0.470834
1925,24-11-2021_07-53-46_AM.jpg,4,0.919752,0.524610,0.552344,0.397916,0.522222


### Объединение csv-файлов в один

In [4]:
import pandas as pd
import os
import shutil

In [36]:
pd_face = pd.read_csv('path/sample_solution_face.csv', delimiter=';')
pd_car = pd.read_csv('path/sample_solution_cars.csv', delimiter=';')
pd_carplate = pd.read_csv('path/sample_solution_carplates.csv', delimiter=';')
pd_human = pd.read_csv('path/sample_solution_human.csv', delimiter=';')
pd_head = pd.read_csv('psth/sample_solution_head.csv', delimiter=';')

In [38]:
pd_res = pd.concat([pd_face, pd_car, pd_carplate, pd_human, pd_head])

In [49]:
pd_res.to_csv('/content/drive/MyDrive/data_sec/data_for_yolo/outputs/result.csv', index=False, sep=';')

In [10]:
pd_res_2 = pd.read_csv('/content/drive/MyDrive/data_sec/data_for_yolo/outputs/result.csv', delimiter=';')

In [11]:
pd_res_2.LabelName.value_counts()

0    7317
3    2045
1    1759
2     485
4     222
Name: LabelName, dtype: int64